In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds = open_mdsdataset(data_dir, geometry='cartesian', endian='<')

/project/6001902/jxchang/Ex3/input/


IndexError: list index out of range

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seawater as sw
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
import matplotlib.animation as animation

In [ ]:
t = 0
h0 = 340.
H = float(currentDirectory[-11:-7])
N0 = float(currentDirectory[-16:-12])/1e6
Fr=int(currentDirectory[-22:-18])/1e4
U0=Fr*N0*h0
f0 = 1.e-4
g = 9.8

print('U = ' + str(U0))
print('Fr = ' + str(Fr))
print('N0 = ' + str(N0))

om=2*np.pi/12.4/3600 
alpha = 2e-4 
nz = 200 
dz=H/nz 
Tref=35+np.cumsum(N0**2/g/alpha*(-H)) 

xmin = 40
xmax = 75
vmax = 3
vmin = -3
wmax = 0.2
wmin = -0.2
tmax = 35
tmin = floor(Tref) 
numcolt=21
numcolv=30

ttlen=len(ds.time)
print('the length of time:' str(ttlen) )
print('initial temp: ' Tref) 
print(tmin)

In [ ]:
#plot bathymetry
fig, axs = plt.subplots(nrows=3, ncols=1,figsize=(20,30))
fig.subplots_adjust(hspace=0.3)
#fig.suptitle("t", fontsize=12)
axs = axs.flatten()


# plot temperature
cmap = cm.RdBu_r
levels = np.linspace(tmin, tmax, num=numcolt)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

T0 = np.squeeze(ds.THETA.values[0, :, :, :])
THETA = np.squeeze(ds.THETA.values[t, :, :, :])
THETA = np.ma.masked_where(T0 == 0, THETA)
N = sw.bfrq(35,T0[:,0],ds.THETA.Z)
X, Z = np.meshgrid(ds.THETA.XC, ds.THETA.Z)

cset1 = axs[0].contourf(X/1e3, Z, THETA, levels, **opts)
fig.colorbar(cset1, ax=axs[0],ticks=np.arange(tmin, tmax+0.1, 5))
axs[0].set_xlabel('X [km]')
axs[0].set_ylabel('Z [m]')
axs[0].set_title('T [oC]', loc='left')
axs[0].set(xlim=(xmin, xmax))
line = axs[0].plot(ds.Depth.XC.values/1e3,-np.transpose(ds.Depth.values),color='black')[0]

In [ ]:
#plot u velocity

UVEL = np.squeeze(ds.UVEL.values[t, :, :, :])
UVEL = np.ma.masked_where(T0 == 0, UVEL)
Xu, Zu = np.meshgrid(ds.UVEL.XG, ds.UVEL.Z)
points_u = np.transpose([Xu.flatten()/1e3,Zu.flatten()])
print(np.shape(UVEL))
print(np.shape(UVEL.flatten()))
print(np.shape(ds.UVEL.XG))
print(np.shape(points_u))

In [ ]:
WVEL = np.squeeze(ds.WVEL.values[t, :, :, :])
WVEL = np.ma.masked_where(T0 == 0, WVEL)
Xw, Zw = np.meshgrid(ds.WVEL.XC, ds.WVEL.Zl)
points_w = np.transpose([Xw.flatten()/1e3,Zw.flatten()])

In [ ]:
from scipy.interpolate import griddata
grid_x, grid_y = np.mgrid[0:100:50j,-4000:0:10j]
grid_U = griddata(points_u,UVEL.flatten(),(grid_x, grid_y))
grid_w = griddata(points_w,WVEL.flatten(),(grid_x, grid_y))
print(np.shape(grid_x))
print(np.shape(grid_U))
print(np.shape(grid_w))

In [ ]:
levels1 = np.linspace(vmin, vmax, num=numcolv)
norm = BoundaryNorm(levels1, ncolors=cmap.N, clip=True)
opts1 = {'cmap':cm.get_cmap(cmap, len(levels1) - 1), 'norm': norm}

cset1 = axs[1].contourf(Xu/1e3, Zu, UVEL, levels1, **opts1)
fig.colorbar(cset1, ax=axs[1],ticks=np.arange(vmin, vmax+0.1, 0.5))
axs[1].set_xlabel('X [km]')
axs[1].set_ylabel('Z [m]')
axs[1].set_title('U [m/s] ', loc='left')
axs[1].set(xlim=(xmin, xmax))
line = axs[1].plot(ds.Depth.XC.values/1e3,-np.transpose(ds.Depth.values),color='black')[0]

axs[2].plot(ds.time/3600,np.zeros(ttlen),color='gray',linestyle='--')
axs[2].plot(ds.time/3600,ds.UVEL.values[:, 0, 0, 0],label='U(t,0,0,0)')
axs[2].plot(ds.time/3600,u0*np.cos(om*ds.time),label='Uinput')
axs[2].set_xlim(min(ds.time/3600), max(ds.time/3600))
axs[2].set_ylim(-u0-0.1, u0+0.1)
point, = axs[2].plot([ds.time[0]/3600],[ds.UVEL.values[0, 0, 0, 0]], 'go', color='red',ms=10)
axs[2].set_xlabel('time [hr]')
axs[2].set_ylabel('U [m/s]')
axs[1].set_title('Uinput vs U[t,0,0,0] ', loc='left')


In [ ]:
quiv = axs[0].quiver(grid_x, grid_y, grid_U/u0, grid_w/u0, color='limegreen',scale=10)

In [ ]:
fig
print(axs[0].collections)
print(np.size(axs[0].collections))

In [ ]:
tt: int=0
def updateData(tt):

    U = np.squeeze(ds.UVEL.values[tt,:,:,:])
    U = np.ma.masked_where(T0 == 0, U)
    W = np.squeeze(ds.WVEL.values[tt,:,:,:])
    W = np.ma.masked_where(T0 == 0, W)
    T = np.squeeze(ds.THETA.values[tt,:,:,:])
    T = np.ma.masked_where(T0 == 0, T)
    grid_U = griddata(points_u,U.flatten(),(grid_x, grid_y))
    grid_w = griddata(points_w,W.flatten(),(grid_x, grid_y))
    time = ds.time.values[tt]/3600
    axs[0].collections[0:numcolt-1] = []
    axs[0].contourf(X/1e3, Z, T, levels, **opts)
    print(np.size(axs[0].collections))
    print(axs[0].collections)
    axs[0].collections.append(axs[0].collections.pop(0))
    line.set_ydata(-np.transpose(ds.Depth.values))
    quiv.set_UVC(grid_U/u0,grid_w/u0)
    axs[1].collections = []
    axs[1].contourf(Xu/1e3, Zu, U, levels1, **opts1)
    print(time)
    print(U.shape)
    point.set_data(time,U[0,0])
    axs[0].set_title('t= %2.3fhr' % time)



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= ttlen-1 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_TUvec.gif', writer='imagemagick', fps=2)

In [ ]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_TUvec.gif'))